# ReAct agent
https://python.langchain.com/docs/tutorials/agents/

In [2]:
#%pip install -q langgraph langchain-tavily langgraph-checkpoint-sqlite

In [3]:
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


In [4]:
load_dotenv()

True

Tavily tool definition

In [ ]:
tavily_search = TavilySearch(max_results=2)

search_results = tavily_search.invoke("What is the temperature in Madrid now")
print(search_results)

hugging face llm: https://python.langchain.com/docs/integrations/chat/huggingface/


In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    provider="auto",  # let Hugging Face choose the best provider for you
)

model = ChatHuggingFace(llm=llm)
tools = [tavily_search]
chat_with_tools = model.bind_tools(tools)

In [8]:
user_prompt = "How is the weather today in SF?"
response = chat_with_tools.invoke([{"role": "user", "content": user_prompt}])
print(response)

content="I'll check the current weather conditions in San Francisco for you." additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "San Francisco weather today current conditions", "search_depth": "basic", "time_range": "day"}', 'name': 'tavily_search', 'description': None}, 'id': 'call_0tmX09Llvbkjt1dL43p5IaH1', 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 1792, 'total_tokens': 1840}, 'model_name': 'deepseek-ai/DeepSeek-V3.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--c5d8d13c-fbd3-4138-b6ef-d4e7c74c8882-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'San Francisco weather today current conditions', 'search_depth': 'basic', 'time_range': 'day'}, 'id': 'call_0tmX09Llvbkjt1dL43p5IaH1', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1792, 'output_tokens': 48, 'total_tokens': 1840}


In [9]:
response.pretty_print()

================================== Ai Message ==================================

I'll check the current weather conditions in San Francisco for you.
Tool Calls:
  tavily_search (call_0tmX09Llvbkjt1dL43p5IaH1)
 Call ID: call_0tmX09Llvbkjt1dL43p5IaH1
  Args:
    query: San Francisco weather today current conditions
    search_depth: basic
    time_range: day


In [10]:
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}\n")

Message content: I'll check the current weather conditions in San Francisco for you.

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'San Francisco weather today current conditions', 'search_depth': 'basic', 'time_range': 'day'}, 'id': 'call_0tmX09Llvbkjt1dL43p5IaH1', 'type': 'tool_call'}]



ReAct agent implementation following https://python.langchain.com/docs/tutorials/agents/

In [11]:
agent_executor = create_react_agent(model, tools, prompt="You are a helpful assistant")

In [12]:
input_message = {"role": "user", "content": "How is the weather today in SF?"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

How is the weather today in SF?!
================================== Ai Message ==================================

I'll search for the current weather in San Francisco to get you the latest information.
Tool Calls:
  tavily_search (call_tVp0a6YSz4HyP4U5Ku3oMIrE)
 Call ID: call_tVp0a6YSz4HyP4U5Ku3oMIrE
  Args:
    query: current weather today San Francisco SF
    time_range: day
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather today San Francisco SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, SF", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Colonia San Francisco (San Francisco)', 'region': 'Zacatecas', 'country': 'Mexico', 'lat': 22.3036, 'lon': -101.8206, 'tz_id': 'America/Mexico_City', 'localtime_epoch': 1759994025, 'localtime': '20

Adding memory to enable multiturn conversations

In [13]:
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [20]:
config = {"configurable": {"thread_id": "abc123"}}
response = agent_executor.invoke({"messages": [("user", "Hi, I'm Bob!")]}, config)

print(response["messages"][-1].pretty_print())

================================== Ai Message ==================================

Hi Bob! It's nice to meet you (again!). 

I'm here and ready to help you with whatever you need - whether it's searching for information, answering questions, or anything else I can assist with. What would you like to do today?
None


In [24]:
for step in agent_executor.stream({"messages": [("user", "What is my name?")]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

Your name is Bob! You've introduced yourself as Bob multiple times in this conversation.


# React agent with custom tools
React agent example that sends  Christmas greetings to customers and notify to a fake slack user using custom tools

In [11]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver

In [12]:
@tool
def send_email(email_address: str, message: str):
    """Send an email to an email address."""
    return "email sent to " + email_address + ": " + message


@tool
def slack_integration(slack_user_id: str, message: str):
    """Send an Slack message to an Slack user."""
    return f"""{slack_user_id}!!:{message}"""

Instantiating the agent

In [13]:
system_prompt = """
You are a helpful agent. Your tasks is to send Christmas cards to our clients using the provided email addresses. Using kind and formal language. The message should not be longer that 15 words.

Send a slack message to @myslackuser and inform the result of sending the email. Include a final summary with the operations you did.
"""

model = init_chat_model("command-r7b-12-2024", model_provider="cohere")
tools = [send_email, slack_integration]
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory, prompt=system_prompt)

In [14]:
# dummy thread to save conversation on memory
config = {"configurable": {"thread_id": "xyz123"}}

In [15]:
for step in agent_executor.stream(
        {"messages": [("user", "Send a Chistmas card to foo@example.com and notme@example.com?")]},
        config,
        stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Send a Chistmas card to foo@example.com and notme@example.com?
================================== Ai Message ==================================

I will use the send_email tool to send a Christmas card to foo@example.com and notme@example.com. I will then use the slack_integration tool to inform @myslackuser of the result of sending the email.
Tool Calls:
  send_email (send_email_5dbs3b1bcgbc)
 Call ID: send_email_5dbs3b1bcgbc
  Args:
    email_address: foo@example.com
    message: Wishing you a Merry Christmas
  send_email (send_email_k7j5dsh6peb8)
 Call ID: send_email_k7j5dsh6peb8
  Args:
    email_address: notme@example.com
    message: Wishing you a Merry Christmas
  slack_integration (slack_integration_m9rzym4ss8pw)
 Call ID: slack_integration_m9rzym4ss8pw
  Args:
    slack_user_id: myslackuser
    message: Email sent to foo@example.com and notme@example.com
================================= Tool Mess